In [1]:
import tensorflow as tf

device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [2]:
!pip install pytorch-transformers
!pip install Keras-Preprocessing
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 176.4/176.4 KB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 880.6/880.6 KB 33.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.6/135.6 KB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 62.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.6/10.6 MB 46.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.6/79.6 KB 10.2 MB/s eta 0:00:00
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895259 sha256=bdf20c84737d2388984e6d1bd2f55269d938f2616bc331d6a19c659fc37f778e
  Stored in directory: /root/.cache/pip/wheels/12/1c/3d/46cf06718d63a32ff798a89594b61e7f345ab6b36d909ce033
Successfully built sacremoses
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wh

In [3]:
import torch
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from keras_preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split


from transformers import XLNetModel, XLNetTokenizer, XLNetForSequenceClassification
from transformers import AdamW

from tqdm import tqdm, trange
import pandas as pd
import io
import numpy as np
import matplotlib.pyplot as plt

In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
n_gpu = torch.cuda.device_count()
torch.cuda.get_device_name(0)

'Tesla T4'

In [6]:
# Upload the train file from your local drive
from google.colab import files
uploaded = files.upload()

Saving balancedtest.csv to balancedtest.csv
Saving fulltrain.csv to fulltrain.csv


In [7]:
df = pd.read_csv("fulltrain.csv", names = ['label', 'text'])
test_df = pd.read_csv("balancedtest.csv", names = ['label', 'text'])

In [8]:
print(test_df.shape)
df.shape

(3000, 2)


(48854, 2)

In [9]:
# Create sentence and label lists
from sklearn.model_selection import train_test_split
from nltk.tokenize import sent_tokenize
import nltk
texts = df.text.values
labels = df.label.values - 1

In [10]:
!pip install unidecode
import unidecode
import re
import string

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.9/235.9 KB 9.5 MB/s eta 0:00:00


In [11]:
nltk.download('punkt')
def preprocess_text(text_arr):
    output_text = []
    for x in text_arr:
        processed = str(unidecode.unidecode(re.sub('[%s]' % re.escape(string.punctuation), '' , x))).lower()
        sents = [s + " [SEP] [CLS]" for s in sent_tokenize(processed)]


        output_text.append(' '.join(sents))
        
    return output_text
sentences = preprocess_text(texts)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [12]:
tokenizer = XLNetTokenizer.from_pretrained('xlnet-base-cased', do_lower_case=True)
tokenized_texts = []
for sent in list(sentences):
  x = tokenizer.tokenize(sent)
  tokenized_texts.append(x)
print(tokenized_texts[:10])

[['▁a', '▁little', '▁less', '▁than', '▁a', '▁decade', '▁ago', '▁hockey', '▁fans', '▁were', '▁blessed', '▁with', '▁a', '▁slate', '▁of', '▁games', '▁every', '▁night', '▁but', '▁on', '▁', 'th', 'ur', 's', 'day', '▁sources', '▁confirmed', '▁that', '▁for', '▁the', '▁ninth', '▁consecutive', '▁year', '▁', 'n', 'hl', '▁players', '▁have', '▁been', '▁locked', '▁out', '▁with', '▁very', '▁slim', '▁hopes', '▁of', '▁an', '▁agreement', '▁in', '▁sight', '▁it', '▁seems', '▁like', '▁just', '▁yesterday', '▁', 'mar', 'tin', '▁', 'st', '▁', 'lou', 'is', '▁and', '▁his', '▁lightning', '▁teammates', '▁were', '▁raising', '▁the', '▁', 'stan', 'ley', '▁cup', '▁high', '▁school', '▁hockey', '▁coach', '▁and', '▁one', 'time', '▁', 'es', 'p', 'n', '▁analyst', '▁bar', 'ry', '▁', 'mel', 'rose', '▁said', '▁obviously', '▁im', '▁still', '▁hoping', '▁the', '▁two', '▁sides', '▁can', '▁come', '▁together', '▁and', '▁reach', '▁an', '▁agreement', '▁but', '▁im', '▁starting', '▁to', '▁think', '▁nobody', '▁really', '▁misses', '▁ho

In [13]:
# Set the maximum sequence length. The longest sequence in our training set is 47, but we'll leave room on the end anyway. 
MAX_LEN = 128

In [14]:
# Use the XLNet tokenizer to convert the tokens to their index numbers in the XLNet vocabulary
input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]

In [15]:
# Pad our input tokens
input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")

In [16]:
# Create attention masks
attention_masks = []

# Create a mask of 1s for each token followed by 0s for padding
for seq in input_ids:
  seq_mask = [float(i>0) for i in seq]
  attention_masks.append(seq_mask)

In [17]:
# Use train_test_split to split our data into train and validation sets for training

train_inputs, validation_inputs, train_labels, validation_labels = train_test_split(input_ids, labels, 
                                                            random_state=2018, test_size=0.1)
train_masks, validation_masks, _, _ = train_test_split(attention_masks, input_ids,
                                             random_state=2018, test_size=0.1)

In [18]:
# Convert all of our data into torch tensors, the required datatype for our model

train_inputs = torch.tensor(train_inputs)
validation_inputs = torch.tensor(validation_inputs)
train_labels = torch.tensor(train_labels)
validation_labels = torch.tensor(validation_labels)
train_masks = torch.tensor(train_masks)
validation_masks = torch.tensor(validation_masks)

In [19]:
# Select a batch size for training. For fine-tuning with XLNet, the authors recommend a batch size of 32, 48, or 128. We will use 32 here to avoid memory issues.
batch_size = 32

# Create an iterator of our data with torch DataLoader. This helps save on memory during training because, unlike a for loop, 
# with an iterator the entire dataset does not need to be loaded into memory

train_data = TensorDataset(train_inputs, train_masks, train_labels)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

validation_data = TensorDataset(validation_inputs, validation_masks, validation_labels)
validation_sampler = SequentialSampler(validation_data)
validation_dataloader = DataLoader(validation_data, sampler=validation_sampler, batch_size=batch_size)


In [20]:
# Load XLNEtForSequenceClassification, the pretrained XLNet model with a single linear classification layer on top. 

model = XLNetForSequenceClassification.from_pretrained("xlnet-base-cased", num_labels=4)
model.cuda()

Some weights of the model checkpoint at xlnet-base-cased were not used when initializing XLNetForSequenceClassification: ['lm_loss.bias', 'lm_loss.weight']
- This IS expected if you are initializing XLNetForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLNetForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLNetForSequenceClassification were not initialized from the model checkpoint at xlnet-base-cased and are newly initialized: ['sequence_summary.summary.bias', 'logits_proj.weight', 'logits_proj.bias', 'sequence_summary.summary.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions a

XLNetForSequenceClassification(
  (transformer): XLNetModel(
    (word_embedding): Embedding(32000, 768)
    (layer): ModuleList(
      (0-11): 12 x XLNetLayer(
        (rel_attn): XLNetRelativeAttention(
          (layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (ff): XLNetFeedForward(
          (layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (layer_1): Linear(in_features=768, out_features=3072, bias=True)
          (layer_2): Linear(in_features=3072, out_features=768, bias=True)
          (dropout): Dropout(p=0.1, inplace=False)
          (activation_function): GELUActivation()
        )
        (dropout): Dropout(p=0.1, inplace=False)
      )
    )
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (sequence_summary): SequenceSummary(
    (summary): Linear(in_features=768, out_features=768, bias=True)
    (activation): Tanh()
    (first_dropout): Identity()
    (last

In [21]:

param_optimizer = list(model.named_parameters())
no_decay = ['bias', 'gamma', 'beta']
optimizer_grouped_parameters = [
    {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)],
     'weight_decay_rate': 0.01},
    {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)],
     'weight_decay_rate': 0.0}
]



In [22]:
# This variable contains all of the hyperparemeter information our training loop needs
optimizer = AdamW(optimizer_grouped_parameters,
                     lr=2e-5)

/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [23]:
# Function to calculate the accuracy of our predictions vs labels
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

In [24]:
out = []
# Store our loss and accuracy for plotting
train_loss_set = []
  
# Training

# Set our model to training mode (as opposed to evaluation mode)
model.train()

# Tracking variables
tr_loss = 0
nb_tr_examples, nb_tr_steps = 0, 0

# Train the data for one epoch
for step, batch in enumerate(train_dataloader):
  # Add batch to GPU
  batch = tuple(t.to(device) for t in batch)
  # Unpack the inputs from our dataloader
  b_input_ids, b_input_mask, b_labels = batch
  # Clear out the gradients (by default they accumulate)
  optimizer.zero_grad()
  # Forward pass
  outputs = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask, labels=b_labels)
  loss = outputs[0]
  logits = outputs[1]
  train_loss_set.append(loss.item())    
  # Backward pass
  loss.backward()
  # Update parameters and take a step using the computed gradient
  optimizer.step()
  
  
  # Update tracking variables
  tr_loss += loss.item()
  nb_tr_examples += b_input_ids.size(0)
  nb_tr_steps += 1

print("Train loss: {}".format(tr_loss/nb_tr_steps))

Train loss: 0.15452618579250751


In [25]:
# Validation

# Put model in evaluation mode to evaluate loss on the validation set
model.eval()

# Tracking variables 
eval_loss, eval_accuracy = 0, 0
nb_eval_steps, nb_eval_examples = 0, 0
eval_f1 = 0

tp = [0, 0, 0, 0]
fp = [0, 0, 0, 0]
fn = [0, 0, 0, 0]

# Evaluate data for one epoch
for batch in validation_dataloader:
  # Add batch to GPU
  batch = tuple(t.to(device) for t in batch)
  # Unpack the inputs from our dataloader
  b_input_ids, b_input_mask, b_labels = batch
  # Telling the model not to compute or store gradients, saving memory and speeding up validation
  with torch.no_grad():
    # Forward pass, calculate logit predictions
    output = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask)
    logits = output[0]
  
  # Move logits and labels to CPU
  logits = logits.detach().cpu().numpy()
  label_ids = b_labels.to('cpu').numpy()

  tmp_eval_accuracy = flat_accuracy(logits, label_ids)
  # tmp_f1 = flat_f1(logits, label_ids)
  
  eval_accuracy += tmp_eval_accuracy
  # eval_f1 += tmp_f1
  nb_eval_steps += 1

  pred_flat = np.argmax(logits, axis=1).flatten()
  labels_flat = label_ids.flatten()
  for i in range(4):
      ai = [i] * len(labels_flat)
      fp[i] += np.sum((pred_flat != labels_flat) & (pred_flat == ai))
      tp[i] += np.sum((pred_flat == labels_flat) & (pred_flat == ai))
      fn[i] += np.sum((pred_flat != labels_flat) & (labels_flat == ai))

f1_total = 0
for i in range(4):
  if tp[i] == 0:
    print("0 tp -", i)
    continue

  p = tp[i] / (tp[i] + fp[i])
  r = tp[i] / (tp[i] + fn[i])
  f1_total += 2*(p * r) / (p + r)
  
print("Validation Accuracy: {}".format(eval_accuracy/nb_eval_steps))
print("Validation Macro F1: {}".format(f1_total/4))

Validation Accuracy: 0.9771241830065359
Validation Macro F1: 0.9747002085552463


In [36]:
test_texts = test_df.text.values
test_labels = test_df.label.values - 1
test_sentences = preprocess_text(test_texts)
test_tokenized_texts = []
for sent in list(test_sentences):
  x = tokenizer.tokenize(sent)
  test_tokenized_texts.append(x)

test_input_ids = [tokenizer.convert_tokens_to_ids(x) for x in test_tokenized_texts]
test_input_ids = pad_sequences(test_input_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")
test_attention_masks = []

# Create a mask of 1s for each token followed by 0s for padding
for seq in test_input_ids:
  seq_mask = [float(i>0) for i in seq]
  test_attention_masks.append(seq_mask)

# _, test_validation_inputs, _, test_validation_labels = train_test_split(test_input_ids, test_labels, 
#                                                             random_state=2018, test_size=1)
# _, test_validation_masks, _, _ = train_test_split(test_attention_masks, test_input_ids,
#                                              random_state=2018, test_size=1)

test_validation_inputs = torch.tensor(test_input_ids)
test_validation_labels = torch.tensor(test_labels)
test_validation_masks = torch.tensor(test_attention_masks)
test_validation_data = TensorDataset(test_validation_inputs, test_validation_masks, test_validation_labels)
test_validation_sampler = SequentialSampler(test_validation_data)
test_validation_dataloader = DataLoader(test_validation_data, sampler=test_validation_sampler, batch_size=batch_size)

In [27]:
model.eval()

XLNetForSequenceClassification(
  (transformer): XLNetModel(
    (word_embedding): Embedding(32000, 768)
    (layer): ModuleList(
      (0-11): 12 x XLNetLayer(
        (rel_attn): XLNetRelativeAttention(
          (layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (ff): XLNetFeedForward(
          (layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (layer_1): Linear(in_features=768, out_features=3072, bias=True)
          (layer_2): Linear(in_features=3072, out_features=768, bias=True)
          (dropout): Dropout(p=0.1, inplace=False)
          (activation_function): GELUActivation()
        )
        (dropout): Dropout(p=0.1, inplace=False)
      )
    )
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (sequence_summary): SequenceSummary(
    (summary): Linear(in_features=768, out_features=768, bias=True)
    (activation): Tanh()
    (first_dropout): Identity()
    (last

In [35]:
type(test_validation_inputs)

torch.Tensor

In [ ]:
eval_loss, eval_accuracy = 0, 0
nb_eval_steps, nb_eval_examples = 0, 0

tp = [0, 0, 0, 0]
fp = [0, 0, 0, 0]
fn = [0, 0, 0, 0]


for batch in test_validation_dataloader:
  # Add batch to GPU
  batch = tuple(t.to(device) for t in batch)
  # Unpack the inputs from our dataloader
  b_input_ids, b_input_mask, b_labels = batch
  # Telling the model not to compute or store gradients, saving memory and speeding up validation
  with torch.no_grad():
    # Forward pass, calculate logit predictions
    output = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask)
    logits = output[0]
  
  # Move logits and labels to CPU
  logits = logits.detach().cpu().numpy()
  label_ids = b_labels.to('cpu').numpy()

  tmp_eval_accuracy = flat_accuracy(logits, label_ids)
  
  eval_accuracy += tmp_eval_accuracy
  nb_eval_steps += 1

  pred_flat = np.argmax(logits, axis=1).flatten()
  labels_flat = label_ids.flatten()
  for i in range(4):
      ai = [i] * len(labels_flat)
      fp[i] += np.sum((pred_flat != labels_flat) & (pred_flat == ai))
      tp[i] += np.sum((pred_flat == labels_flat) & (pred_flat == ai))
      fn[i] += np.sum((pred_flat != labels_flat) & (labels_flat == ai))

f1_total = 0
for i in range(4):
  if tp[i] == 0:
    print("0 tp -", i)
    continue

  p = tp[i] / (tp[i] + fp[i])
  r = tp[i] / (tp[i] + fn[i])
  f1_total += 2*(p * r) / (p + r)
  
print("Test Validation Accuracy: {}".format(eval_accuracy/nb_eval_steps))
print("Test Validation Macro F1: {}".format(f1_total/4))